In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from tqdm import tqdm
from sentence_transformers import SentenceTransformer

In [2]:
df = pd.read_csv("C:/Training/Kaggle/Datasets/Netflix_TV_Shows_Movies/titles.csv")
df.shape

(5850, 15)

In [5]:
df_indian_movies = df[(df['production_countries'].str.contains('IN')) & (df['type']=="MOVIE" )]
df_indian_movies

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
19,tm94651,Dostana,MOVIE,Two close friends decide to enter law enforcem...,1980,NaN,161,"['drama', 'comedy', 'crime', 'romance', 'action']",['IN'],NaN,tt0080653,2.1,25.0,3.980,4.9
21,tm100027,Alibaba Aur 40 Chor,MOVIE,The movie is based on the story of Ali Baba an...,1979,NaN,146,"['drama', 'romance', 'action', 'fantasy', 'fam...","['SU', 'IN']",NaN,tt0079749,6.2,565.0,2.582,6.8
25,tm356209,Singapore,MOVIE,"When his employee disappears in Singapore, Shy...",1960,NaN,158,"['drama', 'thriller', 'crime']",['IN'],NaN,tt0268639,6.4,84.0,0.792,6.3
29,tm336403,Khoon Khoon,MOVIE,As a crazed killer blazes a trail of blood thr...,1973,NaN,131,"['crime', 'thriller']",['IN'],NaN,tt0230412,5.1,41.0,1.948,3.8
30,tm107372,Manoranjan,MOVIE,Havaldar Ratan is a rookie at the local police...,1974,NaN,162,"['romance', 'drama', 'comedy']",['IN'],NaN,tt0071811,6.5,237.0,1.532,5.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5766,tm1004011,Time to Dance,MOVIE,When a ballroom dancer’s shot at a crucial tou...,2021,NaN,2,"['romance', 'drama']",['IN'],NaN,tt8622232,2.2,963.0,1.043,4.0
5788,tm987730,Tribhanga,MOVIE,"When her estranged mother falls into a coma, a...",2021,NaN,95,"['drama', 'family']",['IN'],NaN,tt11102314,6.2,2922.0,4.294,7.0
5795,tm988613,Madam Chief Minister,MOVIE,A political-drama where a young woman from a s...,2021,NaN,123,['drama'],['IN'],NaN,tt13773882,4.8,1769.0,2.301,6.7
5821,tm1110241,Kaaval,MOVIE,Thampan and Antony are a long time best friend...,2021,NaN,148,"['thriller', 'action', 'drama']",['IN'],NaN,tt11182984,5.1,1497.0,1.978,5.3


In [6]:
df_indian_movies['description'].isnull().sum()

0

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2',
                            token="<token_here>")

c:\Python\anaconda3\envs\hf\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
all_desc = list( df_indian_movies['description'] )
out_embs = []
for r in tqdm(all_desc):
    embd = model.encode(r)
    out_embs.append(embd)
out_embs = np.array(out_embs)
out_embs.shape

100%|██████████| 577/577 [00:39<00:00, 14.66it/s]


(577, 384)

In [41]:
query = "mughal emperor falling in love"
qy_emb = model.encode([query])
qy_emb.shape

(1, 384)

In [42]:
sims = []
for e in tqdm(out_embs):
    sims.append(model.similarity(qy_emb,e).numpy()[0])


100%|██████████| 577/577 [00:00<00:00, 3709.00it/s]


In [43]:
df_ind = df_indian_movies[['id', 'title','description']]
df_ind['sims'] = np.array(sims)

C:\Users\Sanjay Sane\AppData\Local\Temp\ipykernel_23160\1398510999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind['sims'] = np.array(sims)


In [45]:
df_ind.sort_values('sims', ascending=False)

,id,title,description,sims
412,tm29269,Jodhaa Akbar,Jodhaa Akbar is a sixteenth century love story...,0.519925
4023,tm455161,Panipat,"During 18th century India, the Marathas emerge...",0.445302
4341,tm935388,Ginny Weds Sunny,Delhi girl Ginny lives with her matchmaker mot...,0.439301
176,tm12876,Bombay,A Hindu man and a Muslim woman fall in love in...,0.434673
86,tm172482,Sohni Mahiwal,Shahjada Ijjat Beg comes to India with his car...,0.428184
...,...,...,...,...
4472,tm832463,Mundina Nildana,Three different people with three different go...,-0.057429
1444,tm208713,Gabbar Is Back,A vigilante network taking out corrupt officia...,-0.061983
573,tm130301,Gangaajal,"Posted to a small, crime-ridden town, a cop so...",-0.081905
5674,tm886193,Laabam,The president of a farmers' association wants ...,-0.105985


In [49]:
query = "husband finds his wife as a killer"
qy_emb = model.encode([query])
sims = []
for e in tqdm(out_embs):
    sims.append(model.similarity(qy_emb,e).numpy()[0])

df_ind = df_indian_movies[['id', 'title','description']]
df_ind.loc[:,'sims'] = np.array(sims)
df_ind.sort_values('sims', ascending=False)[:5]

100%|██████████| 577/577 [00:00<00:00, 1775.96it/s]
C:\Users\Sanjay Sane\AppData\Local\Temp\ipykernel_23160\3233019787.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind.loc[:,'sims'] = np.array(sims)


,id,title,description,sims
4067,tm886214,Mrs. Serial Killer,When her husband is framed and imprisoned for ...,0.577290
5622,tm867681,Haseen Dillruba,Under investigation as a suspect in her husban...,0.561161
3922,tm918043,Raat Akeli Hai,"When a newly married landlord is murdered, a m...",0.510396
2864,tm374395,Vanjagar Ulagam,A woman is murdered and her neighbour is picke...,0.502987
5355,tm990835,The Girl on the Train,When a voyeuristic divorcee fixates on the liv...,0.487210


In [50]:
def search_similar( query ):
    qy_emb = model.encode([query])
    sims = []
    for e in tqdm(out_embs):
        sims.append(model.similarity(qy_emb,e).numpy()[0])

    df_ind = df_indian_movies[['id', 'title','description']]
    df_ind.loc[:,'sims'] = np.array(sims)
    return df_ind.sort_values('sims', ascending=False)[:5]

In [51]:
search_similar( "three college friends do a lot of fun" )

100%|██████████| 577/577 [00:00<00:00, 1828.95it/s]
C:\Users\Sanjay Sane\AppData\Local\Temp\ipykernel_23160\1937176558.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind.loc[:,'sims'] = np.array(sims)


,id,title,description,sims
434,tm61056,Dil Chahta Hai,Three inseparable childhood friends are just o...,0.612664
721,tm182125,Zindagi Na Milegi Dobara,Three friends who were inseparable in childhoo...,0.507088
997,tm182322,Kai Po Che!,Three friends growing up in India at the turn ...,0.482502
5154,tm1213009,Dear Friend,Four close friends who live together in Bangal...,0.472048
972,tm185797,Fukrey,The paths of four dream-chasing college friend...,0.469725


### Inferencing Interface

In [52]:
import gradio as gr

In [ ]:
demo = gr.Interface(fn = search_similar, 
                    inputs=[gr.Textbox(label="Your Review:")],
                    outputs=[gr.Dataframe(label="Most Similar Description Movies:")])
demo.launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


100%|██████████| 577/577 [00:00<00:00, 1374.42it/s]
C:\Users\Sanjay Sane\AppData\Local\Temp\ipykernel_23160\1937176558.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ind.loc[:,'sims'] = np.array(sims)
